In [ ]:
# =============================
# 1. Setup & Imports
# =============================
import os
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import zipfile

from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

# =============================
# 2. Extract Dataset
# =============================
zip_path = "/content/archive (3).zip"   # change if your zip has diff name
extract_dir = "/content/archive"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Dataset extracted to:", extract_dir)

# =============================
# 3. Load Images
# =============================
import cv2

IMG_SIZE = 64

def load_images_from_folder(folder, img_size):
    data = []
    for img_path in glob(os.path.join(folder, "*")):
        try:
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (img_size, img_size))
            data.append(img)
        except:
            pass
    return np.array(data)

# Target folders
folders = [
    os.path.join(extract_dir, "Kolam19 Images/Kolam19 Images"),
    os.path.join(extract_dir, "Kolam29 Images/Kolam29 Images"),
    os.path.join(extract_dir, "Kolam109 Images/Kolam109 Images")
]

images = []
for f in folders:
    images.extend(load_images_from_folder(f, IMG_SIZE))

images = np.array(images)
images = (images.astype(np.float32) - 127.5) / 127.5   # normalize to [-1,1]

print("✅ Dataset shape:", images.shape)

# =============================
# 4. GAN Model
# =============================
img_shape = (IMG_SIZE, IMG_SIZE, 3)
latent_dim = 100

# Generator
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    return model

# Discriminator
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

optimizer = Adam(0.0002, 0.5)

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

generator = build_generator()

z = Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False
valid = discriminator(img)

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

# =============================
# 5. Training
# =============================
def save_imgs(epoch):
    r, c = 3, 3
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5  # scale back to [0,1]

    fig, axs = plt.subplots(r, c, figsize=(6,6))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    plt.savefig(f"/content/kolam_{epoch}.png")
    plt.close()

def train(dataset, epochs, batch_size=32, sample_interval=50):
    half = batch_size // 2
    for epoch in range(epochs):

        # Train Discriminator
        idx = np.random.randint(0, dataset.shape[0], half)
        imgs = dataset[idx]

        noise = np.random.normal(0, 1, (half, latent_dim))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.ones((batch_size, 1))
        g_loss = combined.train_on_batch(noise, valid_y)

        # Print Progress
        print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

        # Save images
        if epoch % sample_interval == 0:
            save_imgs(epoch)

# =============================
# 6. Run Training
# =============================
train(images, epochs=2000, batch_size=32, sample_interval=200)


✅ Dataset extracted to: /content/archive
✅ Dataset shape: (600, 64, 64, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
0 [D loss: 0.8504, acc.: 14.06%] [G loss: 0.9857]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1 [D loss: 0.8499, acc.: 19.01%] [G loss: 0.8900]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
2 [D loss: 0.8458, acc.: 18.75%] [G loss: 0.8115]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
3 [D loss: 0.8400, acc.: 17.47%] [G loss: 0.7433]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
4 [D loss: 0.8426, acc.: 15.17%] [G loss: 0.6785]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
5 [D loss: 0.8580, acc.: 12.52%] [G loss: 0.6244]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
6 [D loss: 0.8693, acc.: 11.13%] [G loss: 0.5739]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
7 [D loss: 0.8979, acc.: 9.69%] [G loss: 0.5318]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
8 [D loss: 0.9225, acc.: 8.58%] [G loss: 0.4950]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
9 [D loss: 0.9537, acc.: 7.70%] [G loss: 0.4617]
1/1 ━━

KeyboardInterrupt: 

In [ ]:
from PIL import Image, ImageDraw

def generate_kolam(n, spacing=70, margin=50, dot_radius=4, stroke_width=3, filename='kolam.png'):
    """
    Generate a kolam-like image using a grid of n x n dots.
    """
    # Calculate image size
    width = margin*2 + spacing*(n-1)
    height = margin*2 + spacing*(n-1)
    img = Image.new('RGB', (width, height), 'white')
    draw = ImageDraw.Draw(img)

    # Compute dot coordinates
    coords = []
    for r in range(n):
        row = []
        for c in range(n):
            x = margin + c * spacing
            y = margin + r * spacing
            row.append((x, y))
            # Draw dot
            draw.ellipse((x-dot_radius, y-dot_radius, x+dot_radius, y+dot_radius), fill='black')
        coords.append(row)

    # Draw horizontal semicircles
    for r in range(n):
        for c in range(n-1):
            x1, y1 = coords[r][c]
            x2, y2 = coords[r][c+1]
            left = x1
            top = y1 - spacing//2
            right = x2
            bottom = y1 + spacing//2
            if r % 2 == 0:
                start, end = 180, 360  # lower semicircle
            else:
                start, end = 0, 180    # upper semicircle
            draw.arc([left, top, right, bottom], start=start, end=end, width=stroke_width, fill='black')

    # Draw vertical semicircles
    for c in range(n):
        for r in range(n-1):
            x1, y1 = coords[r][c]
            x2, y2 = coords[r+1][c]
            left = x1 - spacing//2
            top = y1
            right = x1 + spacing//2
            bottom = y2
            if c % 2 == 0:
                start, end = 90, 270   # left semicircle
            else:
                start, end = -90, 90   # right semicircle
            draw.arc([left, top, right, bottom], start=start, end=end, width=stroke_width, fill='black')

    # Add inner loops for complexity
    for r in range(n-1):
        for c in range(n-1):
            cx = (coords[r][c][0] + coords[r][c+1][0] + coords[r+1][c][0] + coords[r+1][c+1][0]) / 4
            cy = (coords[r][c][1] + coords[r][c+1][1] + coords[r+1][c][1] + coords[r+1][c+1][1]) / 4
            radius = spacing * 0.12
            draw.ellipse((cx-radius, cy-radius, cx+radius, cy+radius), outline='black', width=2)

    img.save(filename)
    print(f"✅ Kolam saved as {filename}")


# ---------- USER INPUT ----------
if __name__ == "__main__":
    n = int(input("Enter number of dots per row/column: "))
    spacing = int(input("Enter spacing between dots (e.g., 50, 60, 70): "))
    margin = int(input("Enter margin size (default 50): ") or 50)
    filename = input("Enter filename to save (default kolam.png): ") or "kolam.png"
    print("\n")
    print("No. of Dots per Row: ",n)
    print("Space between Dots: ",spacing)
    print("Margin Size: ",margin)


    generate_kolam(n, spacing=spacing, margin=margin, filename=filename)


Enter number of dots per row/column: 11
Enter spacing between dots (e.g., 50, 60, 70): 90
Enter margin size (default 50): 70
Enter filename to save (default kolam.png): fig.png


No. of Dots per Row:  11
Space between Dots:  90
Margin Size:  70
✅ Kolam saved as fig.png


In [ ]:
import gradio as gr
from PIL import Image, ImageDraw

def generate_kolam(n, spacing=70, margin=50, dot_radius=4, stroke_width=3):
    """
    Generate a kolam-like image using a grid of n x n dots.
    """
    # Calculate image size
    width = margin*2 + spacing*(n-1)
    height = margin*2 + spacing*(n-1)
    img = Image.new('RGB', (width, height), 'white')
    draw = ImageDraw.Draw(img)

    # Compute dot coordinates
    coords = []
    for r in range(n):
        row = []
        for c in range(n):
            x = margin + c * spacing
            y = margin + r * spacing
            row.append((x, y))
            # Draw dot
            draw.ellipse((x-dot_radius, y-dot_radius, x+dot_radius, y+dot_radius), fill='black')
        coords.append(row)

    # Draw horizontal semicircles
    for r in range(n):
        for c in range(n-1):
            x1, y1 = coords[r][c]
            x2, y2 = coords[r][c+1]
            left = x1
            top = y1 - spacing//2
            right = x2
            bottom = y1 + spacing//2
            if r % 2 == 0:
                start, end = 180, 360  # lower semicircle
            else:
                start, end = 0, 180    # upper semicircle
            draw.arc([left, top, right, bottom], start=start, end=end, width=stroke_width, fill='black')

    # Draw vertical semicircles
    for c in range(n):
        for r in range(n-1):
            x1, y1 = coords[r][c]
            x2, y2 = coords[r+1][c]
            left = x1 - spacing//2
            top = y1
            right = x1 + spacing//2
            bottom = y2
            if c % 2 == 0:
                start, end = 90, 270   # left semicircle
            else:
                start, end = -90, 90   # right semicircle
            draw.arc([left, top, right, bottom], start=start, end=end, width=stroke_width, fill='black')

    # Add inner loops for complexity
    for r in range(n-1):
        for c in range(n-1):
            cx = (coords[r][c][0] + coords[r][c+1][0] + coords[r+1][c][0] + coords[r+1][c+1][0]) / 4
            cy = (coords[r][c][1] + coords[r][c+1][1] + coords[r+1][c][1] + coords[r+1][c+1][1]) / 4
            radius = spacing * 0.12
            draw.ellipse((cx-radius, cy-radius, cx+radius, cy+radius), outline='black', width=2)

    return img


# ----------------- GRADIO UI -----------------
def kolam_ui(n, spacing, margin):
    return generate_kolam(n, spacing=spacing, margin=margin)

with gr.Blocks() as demo:
    gr.Markdown("## 🎨 Kolam Generator\nGenerate symmetric Kolam patterns with dots and curves.")

    with gr.Row():
        n = gr.Slider(3, 15, value=7, step=1, label="Number of Dots per Row/Column")
        spacing = gr.Slider(40, 100, value=70, step=5, label="Spacing Between Dots")
        margin = gr.Slider(20, 100, value=50, step=5, label="Margin Size")

    btn = gr.Button("Generate Kolam")
    output = gr.Image(type="pil", label="Generated Kolam")

    btn.click(fn=kolam_ui, inputs=[n, spacing, margin], outputs=output)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5163a716de4073aa68.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
